In [1]:
import numpy as np
import pandas as pd
from utils.diff import diff

In [2]:
# 13. chato
# 季度频率。销售变化量除以平均总资产。

# 14. chato_ia
# 季度频率。经行业调整的销售变化除以平均总资产。

df1 = pd.read_csv('../../data/csmar/季_利润表.csv', usecols=['证券代码', '会计期间', '营业总收入'])
df1 = diff(df1, col=['营业总收入'])
df1['DTSale'] = df1['营业总收入'] - df1.groupby('证券代码').shift(1)['营业总收入']
df2 = pd.read_csv('../../data/csmar/季_资产负债表.csv', usecols=['证券代码', '会计期间', '资产总计'], low_memory=False)
df2['AveTA'] = (df2['资产总计'] + df2.groupby('证券代码').shift(1)['资产总计'])/2
df3 = pd.merge(df1, df2, on=['证券代码', '会计期间'], how='left')
df3['chato'] = df3['DTSale']/df3['AveTA'].replace(0, np.nan)
df4 = pd.read_csv('../../data/csmar/公司文件.csv', usecols=['证券代码', '行业代码C'])
df5 = pd.merge(df3, df4, on=['证券代码'])
df6 = df5[['行业代码C', '会计期间', 'chato']].groupby(['行业代码C', '会计期间']).mean().reset_index()
df6.rename(columns={'chato':'chato_i'}, inplace=True)
df7 = pd.merge(df5,df6,on=['行业代码C', '会计期间'])
df7['chato_ia'] = df7['chato'] - df7['chato_i']
df8 = df7[['证券代码', '会计期间', 'chato', 'chato_ia']]
df8.to_csv('../../output/csmar/季_13_14.csv', index=False, encoding='utf-8-sig')
df8

,证券代码,会计期间,chato,chato_ia
0,1,1990-03-31,NaN,NaN
1,1227,1990-03-31,NaN,NaN
2,2142,1990-03-31,NaN,NaN
3,2807,1990-03-31,NaN,NaN
4,2839,1990-03-31,NaN,NaN
...,...,...,...,...
728835,605086,2022-09-30,0.013539,0.014077
728836,603505,2022-12-31,0.056040,0.041690
728837,605086,2022-12-31,-0.027340,-0.041690
728838,603505,2023-03-31,-0.070002,-0.039984
